In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, datetime
import datetime

In [0]:
export_timestamp            = date.today().strftime('%Y%m%d')  #datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")

destination_table           = "data_user_menaka.perfect_golden_record_cp_consent"
destination_table_overview  = "data_user_menaka.perfect_golden_record_cp_consent_overview"
created_datetime            = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [0]:
ohub2_cp = spark.sql("""
SELECT cp.countrycode as cp_countryCode,
              cp.ohubid as cp_ohubId,  
              cp.operatorohubid as  cp_operatorOhubId,      
              cp.emailAddress as cp_emailAddress,     
              cp.isEmailAddressValid as cp_isEmailAddressValid,
              
             case when
        ((cp.emailAddress is not null and cp.hasEmailOptIn = true and (cp.hasEmailOptout is null or cp.hasEmailOptout = false) and cp.countrycode not in('AT','DE','CA'))
          or (cp.emailAddress is not null and cp.hasEmailOptIn = true and (cp.hasEmailOptout is null or cp.hasEmailOptout = false) and cp.countrycode in('AT','DE','CA') and cp.hasemaildoubleoptin = true)
          or (cp.emailAddress is not null and cp.hasEmailOptIn is null and cp.hasEmailOptout = false and cp.countrycode not in('AT','BE','BG','HR','CZ','DK','EE','FI','FR','DE','GR','HU','IE','IT','LT','LV','GB','NO','NL','PL','PT','RO','SL','SK','ES','SE','CA','ID'))
          or (cp.emailAddress is not null and cp.isEmailAddressValid = true
            and cp.hasEmailOptIn is null
            and cp.hasEmailOptout is null
            and ((cp.countrycode='TR' and (upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%SSDMICROSITE%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%' or upper(cp.sourcename) like '%KANGAROO%'))
              or (cp.countrycode ='CH' and (upper(cp.sourcename) like '%DEX%' or upper(cp.sourcename) like '%BTG%' or upper(cp.sourcename) like '%MM-INIT-OPER%'))
              or (cp.countrycode in('PH','TH') and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%ARMSTRONG%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%'))
              or (cp.countrycode='ID' and (upper(cp.sourcename) like '%ARMSTRONG%' or upper(cp.sourcename) like '%ASKROSANA%' or upper(cp.sourcename) like '%SSDOTHER%'))
              or (cp.countrycode = 'US' and (upper(cp.sourcename) like '%ANTHEM%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%SSDMICROSITE%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%'))
              or (cp.countrycode='TW' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%KLP%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%'))
              or (cp.countrycode='SA' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%ARMSTRONG%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%TELESALES%' or upper(cp.sourcename) like '%SSDMICROSITE%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%'))
              or (cp.countrycode in('PK','LK','KR') and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%MM-INIT-OPER%'))
              or (cp.countrycode in('PE','NU','LB','BO','AF') and (upper(cp.sourcename) like '%SSDMICROSITE%'))
              or (cp.countrycode='MY' and (upper(cp.sourcename) like '%ARMSTRONG%'))
              or (cp.countrycode='HK' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%KLP%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%'))
              or (cp.countrycode='EG' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%ARMSTRONG%' or upper(cp.sourcename) like '%WEBUPDATER%'))
              or (cp.countrycode in('AE') and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%'))
              or (cp.countrycode in('CL') and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%IOPERA%' or upper(cp.sourcename) like '%EMAKINA%' or upper(cp.sourcename) like '%LIVECHAT%' or upper(cp.sourcename) like '%FACEBOOK%'))
              or (cp.countrycode='BR' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%IOPERA%' or upper(cp.sourcename) like '%SSDMICROSITE%' or upper(cp.sourcename) like '%MM-INIT-OPER%' or upper(cp.sourcename) like '%EMAKINA%'))
              or (cp.countrycode='BH' and (upper(cp.sourcename) like '%MELLOWMESSAGE%'))
              or (cp.countrycode='AZ' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%SSDMICROSITE%'))
              or (cp.countrycode='AR' and (upper(cp.sourcename) like '%MELLOWMESSAGE%' or upper(cp.sourcename) like '%WEBUPDATER%' or upper(cp.sourcename) like '%SSDMICROSITE%' or upper(cp.sourcename) like '%MM-INIT-OPER%'))
            )
          )
        )
        and (cp.hasgeneraloptout = false or cp.hasgeneraloptout is null)
      then 'Y'
    end reachableEmail,
    cp.emailOptInDate,
    cp.emailDoubleOptInDate,
    
    case
      when ((cp.hasgeneraloptout = false or cp.hasgeneraloptout is null) and
        (
          ((cp.emailAddress is not null and cp.isEmailAddressValid = true and cp.hasEmailOptIn = true and (cp.hasEmailOptout is null or cp.hasEmailOptout = false) and cp.countrycode not in('AT','DE','CA','ID'))
            or (cp.emailAddress is not null and cp.isEmailAddressValid = true and cp.hasEmailOptIn = true and (cp.hasEmailOptout is null or cp.hasEmailOptout = false) and cp.countrycode in('AT','DE','CA','ID') and cp.hasemaildoubleoptin = true)
            or (cp.emailAddress is not null and cp.isEmailAddressValid = true and cp.hasEmailOptIn is null and cp.hasEmailOptout = false and cp.countrycode not in('AT','BE','BG','HR','CZ','DK','EE','FI','FR','DE','GR','HU','IE','IT','LT','LV','GB','NO','NL','PL','PT','RO','SL','SK','ES','SE','CA','ID'))
            or (cp.emailAddress is not null and cp.isEmailAddressValid = true and cp.hasEmailOptIn is null and cp.hasEmailOptout is null
              and ((cp.countrycode='TR' and cp.sourcename in('WEBUPDATER','SSD_MICROSITE','MM-INIT-OPER','EMAKINA','KANGAROO'))
                or (cp.countrycode ='CH' and upper(cp.sourcename) in('DEX','BTG','MM-INIT-OPER'))
                or (cp.countrycode in('PH','TH') and upper(cp.sourcename) in('MELLOWMESSAGE','ARMSTRONG','WEBUPDATER','MM-INIT-OPER','EMAKINA'))
                or (cp.countrycode='ID' and upper(cp.sourcename) in('ARMSTRONG','ASKROSANA','SSD_OTHER'))
                or (cp.countrycode='US' and upper(cp.sourcename) in('ANTHEM','WEBUPDATER','SSD_MICROSITE','MM-INIT-OPER','EMAKINA'))
                or (cp.countrycode='TW' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','KLP','MM-INIT-OPER','EMAKINA'))
                or (cp.countrycode='SA' and upper(cp.sourcename) in('MELLOWMESSAGE','ARMSTRONG','WEBUPDATER','TELESALES','SSD_MICROSITE','MM-INIT-OPER','EMAKINA'))
                or (cp.countrycode in('PK','LK','KR') and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','MM-INIT-OPER'))
                or (cp.countrycode in('PE','NU','LB','BO','AF') and upper(cp.sourcename) in('SSD_MICROSITE'))
                or (cp.countrycode='MY' and upper(cp.sourcename) in('ARMSTRONG'))
                or (cp.countrycode='HK' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','KLP','MM-INIT-OPER','EMAKINA'))
                or (cp.countrycode='EG' and upper(cp.sourcename) in('MELLOWMESSAGE','ARMSTRONG','WEBUPDATER'))
                or (cp.countrycode in('CL','AE') and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER'))
                or (cp.countrycode='BR' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','IOPERA','SSD_MICROSITE','MM-INIT-OPER','EMAKINA'))
                or (cp.countrycode='BH' and upper(cp.sourcename) in('MELLOWMESSAGE'))
                or (cp.countrycode='AZ' and upper(cp.sourcename) in('MELLOWMESSAGE','SSD_MICROSITE'))
                or (cp.countrycode='AR' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','SSD_MICROSITE','MM-INIT-OPER')))
            )
          )
            or ((cp.mobilenumber is not null and cp.hasmobileOptIn = true and (cp.hasmobileOptout is null or cp.hasmobileOptout = false) and cp.countrycode not in('AT','DE','CA','ID'))
              or (cp.mobilenumber is not null and cp.hasmobileOptIn=true and (cp.hasmobileOptout is null or cp.hasmobileOptout=false) and cp.countrycode in('AT','DE','CA','ID') and cp.hasmobiledoubleoptin = true)
              or (cp.mobilenumber is not null and cp.hasmobileOptIn is null and cp.hasmobileOptout = false and cp.countrycode not in('AT','BE','BG','HR','CZ','DK','EE','FI','FR','DE','GR','HU','IE','IT','LT','LV','GB','NO','NL','PL','PT','RO','SL','SK','ES','SE','CA','ID'))
              or (cp.mobilenumber is not null and cp.hasmobileOptIn is null and cp.hasmobileOptout is null
                and ((cp.countrycode='TR' and cp.sourcename in('WEBUPDATER','SSD_MICROSITE','MM-INIT-OPER','EMAKINA','KANGAROO'))
                  or (cp.countrycode ='CH' and upper(cp.sourcename) in('DEX','BTG','MM-INIT-OPER'))
                  or (cp.countrycode in('PH','TH') and upper(cp.sourcename) in('MELLOWMESSAGE','ARMSTRONG','WEBUPDATER','MM-INIT-OPER','EMAKINA'))
                  or (cp.countrycode='ID' and upper(cp.sourcename) in('ARMSTRONG','ASKROSANA','SSD_OTHER'))
                  or (cp.countrycode='US' and upper(cp.sourcename) in('ANTHEM','WEBUPDATER','SSD_MICROSITE','MM-INIT-OPER','EMAKINA'))
                  or (cp.countrycode='TW' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','KLP','MM-INIT-OPER','EMAKINA'))
                  or (cp.countrycode='SA' and upper(cp.sourcename) in('MELLOWMESSAGE','ARMSTRONG','WEBUPDATER','TELESALES','SSD_MICROSITE','MM-INIT-OPER','EMAKINA'))
                  or (cp.countrycode in('PK','LK','KR') and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','MM-INIT-OPER'))
                  or (cp.countrycode in('PE','NU','LB','BO','AF') and upper(cp.sourcename) in('SSD_MICROSITE'))
                  or (cp.countrycode='MY' and upper(cp.sourcename) in('ARMSTRONG'))
                  or (cp.countrycode='HK' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','KLP','MM-INIT-OPER','EMAKINA'))
                  or (cp.countrycode='EG' and upper(cp.sourcename) in('MELLOWMESSAGE','ARMSTRONG','WEBUPDATER'))
                  or (cp.countrycode in('CL','AE') and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER'))
                  or (cp.countrycode='BR' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','IOPERA','SSD_MICROSITE','MM-INIT-OPER','EMAKINA'))
                  or (cp.countrycode='BH' and upper(cp.sourcename) in('MELLOWMESSAGE'))
                  or (cp.countrycode='AZ' and upper(cp.sourcename) in('MELLOWMESSAGE','SSD_MICROSITE'))
                  or (cp.countrycode='AR' and upper(cp.sourcename) in('MELLOWMESSAGE','WEBUPDATER','SSD_MICROSITE','MM-INIT-OPER'))
                )
              )
            )
          )
        )
      then 'Y'
   end reachableAny
              
FROM data_datascience_prod.contactpersons_golden cp
WHERE 1=1
AND cp.isGoldenRecord = true 
AND cp.isactive = true 
GROUP BY 1,2,3,4,5,6,7,8,9
""")

In [0]:
ohub2_cp.write.mode("overwrite").saveAsTable(destination_table)

In [0]:
consent_overview  = ohub2_cp.groupby("cp_countryCode").agg(countDistinct(when(col('reachableEmail') == 'Y', col("cp_ohubId"))).alias("NoofReachableCP"),                                                          
                                                           countDistinct(when( (col('reachableEmail') == 'Y') & (col("cp_isEmailAddressValid") =='true'), col("cp_ohubId"))).alias("NoofReachableValidCP"),
                                                           countDistinct(when(col('reachableEmail') == 'Y', col("cp_emailAddress"))).alias("NoofReachableEmailIds"),
                                                           countDistinct(when( (col('reachableEmail') == 'Y') & (col("cp_isEmailAddressValid") =='true'), col("cp_emailAddress"))).alias("NoofReachableValidEmailIds")
                                                          ).sort("cp_countryCode")
consent_overview  = consent_overview.withColumn("createdDateTime", lit(created_datetime))
#display(consent_overview)

In [0]:
consent_overview.write.mode("overwrite").saveAsTable(destination_table_overview)